## Optimaze the Linkedin parsing code 

In [39]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import random, os, sys, time
import csv
import pandas as pd

In [40]:
## First we need a browser that we will run the selenium library
browser = webdriver.Chrome()

In [41]:
## Next let's Open the page that we will scrape in our case we will scrape the linkedin platform 
# Following the step we are in the sign in page of linkedin and it's time to insert our personal informations
browser.get('https://www.linkedin.com/login')

In [42]:
## In this step I will read a file that contains my informations in order to keep them private
file = open("info.txt")
lines = file.readlines()
username = lines[0]
password = lines[1]

In [43]:
## Let's sign in
# Find the username box and add your username
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

In [44]:
# Find the password box and add your password
elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

In [45]:
## click to the aggre and join option
elementID.submit()

In [46]:
## Determine the link of the search  
link = 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL'

In [47]:
## Get the link 
browser.get(link)
time.sleep(2)

In [48]:
## Now we are in the page that we want to scrape
## I will use the beautiful soup library to scrape the page
# Our goal is to parse all the companies with some details that will be useful for the competition analysis 
page = bs(browser.page_source, "html.parser")

In [49]:
## Creating a function in order to collect the informations that we need from the pages

def get_features(card):
    href = card.a
    title = card.find("span", "entity-result__title-line flex-shrink-1 entity-result__title-text--black").text.replace("\n","")
    link = href.get("href")
    try:
        sector = card.find("div", "entity-result__primary-subtitle t-14 t-black").text.replace("\n", "")
    except:
        sector = "None"
    followers = card.find("div", "entity-result__secondary-subtitle t-14").text.replace("\n","")
    try:
        description = card.find("p", "entity-result__summary t-12 t-black--light mb1").text.replace("\n","")
    except:
        description = "None"
    try:  
        current_positions = card.find("span", "entity-result__simple-insight-text").text.replace("\n","")
    except:
        current_positions = "None"
        
    feature = (title, link, sector, followers, description, current_positions)    
    return feature

In [50]:
## Finding the total number of the pages that will scrape informations from
ul = page.find("ul", "artdeco-pagination__pages artdeco-pagination__pages--number")
elements = ul.find_all("li")
len(elements)

10

In [51]:
## append the different url's to a list
pages = []
pages.append(link)  
new_url = 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL' +'&page={num:.2f}'
for i in range(1,len(elements)+2):
    try:
        pages.append(new_url.format(num=i))
    except AttributeError:
        break

In [64]:
pages

['https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=1.00',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=2.00',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=3.00',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=4.00',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=5.00',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruiting&origin=SWITCH_SEARCH_VERTICAL&page=6.00',
 'https://www.linkedin.com/search/results/companies/?keywords=staffing%20%26%20recruit

In [52]:
final_list = []
for link in pages:
    browser.get(link)
    page = bs(browser.page_source, "html.parser")
    #bs.find_all("div", "entity-result")
    time.sleep(2)

    cards = page.find_all("div", "entity-result")
    for card in cards:
            features = get_features(card)
            final_list.append(features)
    time.sleep(2)

In [54]:
len(final_list)

93

In [55]:
## Convert the list to csv file
import csv
with open('new_oper.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["title", "link", "sector", "followers", "description", "current_positions"])
        writer.writerows(final_list)

In [63]:
df = pd.read_csv("new_oper.csv")
df.shape

(93, 6)

In [62]:
## Remove duplicate 
new_df = df.drop_duplicates(subset=['title'])
new_df.shape

(83, 6)